In [1]:
# dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [2]:
# pip install splinter

In [ ]:
# splinter setup
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


In [4]:
# redplanetscience.com 
url = "https://redplanetscience.com/"
browser.visit(url)
time.sleep(1)

In [5]:
#page to soup
html = browser.html
soup = bs(html, "html.parser")

In [6]:
#latest news title
news_title = soup.find('div', class_='content_title').text
# news_title

In [7]:
#latest paragraph of news 
news_p = soup.find('div', class_='article_teaser_body').text
# news_p

In [8]:
#image site 
jpl_url = "https://spaceimages-mars.com/"
browser.visit(jpl_url)
time.sleep(1)

In [9]:
# page into soup
html = browser.html
soup = bs(html, "html.parser")

In [10]:
#navigation with splinter and have url be a variable
relative_image_path = soup.find_all('img')[1]["src"]
featured_image_url = jpl_url + relative_image_path
# featured_image_url

In [11]:
# galaxyfacts
mars_url = "https://galaxyfacts-mars.com/"
tables = pd.read_html(mars_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [12]:
type(tables) #what form the data is in

list

In [13]:
#index normalization
facts_df = tables[0]
#no earth allowed
facts_df = facts_df.drop(columns=[2])
#new names
facts_df = facts_df.rename(columns={0: "Mars Profile", 1: "Measurements"})

# Display DataFrame
facts_df

,Mars Profile,Measurements
0,Mars - Earth Comparison,Mars
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"
5,Length of Year:,687 Earth days
6,Temperature:,-87 to -5 °C


In [14]:
facts_df = facts_df.drop([0])
facts_df

,Mars Profile,Measurements
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"
5,Length of Year:,687 Earth days
6,Temperature:,-87 to -5 °C


In [15]:
#conversion to string for HTML
mars_facts = facts_df.to_html()
mars_facts.replace('\n', '')
# print(mars_facts)
# mars_facts

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars Profile</th>      <th>Measurements</th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>Diameter:</td>      <td>6,779 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2</td>    </tr>    <tr>      <th>4</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>    </tr>    <tr>      <th>5</th>      <td>Length of Year:</td>      <td>687 Earth days</td>    </tr>    <tr>      <th>6</th>      <td>Temperature:</td>      <td>-87 to -5 °C</td>    </tr>  </tbody></table>'

In [16]:
#visit mars hemi
mars_hem_url = "https://marshemispheres.com/"
browser.visit(mars_hem_url)
time.sleep(1)
#to soup
html = browser.html
soup = bs(html, "html.parser")

In [17]:
# link up then extract
links = browser.find_by_css('a.product-item img')
mars_items = soup.find_all('div', class_='item')

In [19]:
#create a dictionary with the hemi-url
hem_img_urls=[]
#loop through mars items 
for item in mars_items:
    #for errors 
    try:
        # take the titles then the url
        hem = item.find('div',class_='description')
        hem_title=hem.h3.text
        hem_url = hem.a['href']
        browser.visit(mars_hem_url + hem_url)
        
        # extrat links for the images 
        html=browser.html
        soup=bs(html,'html.parser')
        img_src=soup.find('li').a['href']
        if (hem_title and img_src):
            print('-'*25)
            print(hem_title)
            print(mars_hem_url+img_src)
            #dict for the urls 
        hem_dict={
            'title': hem_title,
            'image_url': img_src
        }
        hem_img_urls.append(hem_dict)
    except Exception as e:
        print(e)

-------------------------
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
-------------------------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
-------------------------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
-------------------------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


In [20]:
#complete dict for all the data
mars_dict={
    'hem_title': hem_title,
    'hem_image_url': img_src,
    'news_title': news_title,
    'news_p': news_p,
    'featured_img': featured_image_url,
    'mars_facts': mars_facts
    
}